In [69]:
#importing libraries
import requests
import json
import re
import csv

In [70]:
#Initiating session
session = requests.session()

In [71]:
#Redirecting to store-page
res_main = session.get("https://www.lenskart.com/stores")

In [72]:

# Define the pattern
ptrn = 'slug\":\"(.*?)\"'

# Use re.findall to find all matches in the text
matches = re.findall(ptrn, res_main.text)

# Filter out items starting with "optometrist" or "lenskart"
filtered_matches = []
for match in matches:
    if not match.startswith("optometrist") and not match.startswith("lenskart"):
        filtered_matches.append(match)

# Print the filtered matches
print(filtered_matches)



['andhra-pradesh', 'andmaan-nicobar', 'arunachal-pradesh', 'assam', 'bihar', 'chandigarh', 'chennai', 'chhattisgarh', 'dadra-and-nagar-haveli', 'delhi', 'goa', 'gujarat', 'gwalior', 'haryana', 'himachal-pradesh', 'hyderabad', 'jammu-and-kashmir', 'jharkhand', 'karnataka', 'kerala', 'madhya-pradesh', 'maharashtra', 'manipur', 'nagaland', 'new-delhi', 'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim', 'tamil-nadu', 'tamil\xa0nadu', 'telangana', 'tripura', 'uttar-pradesh', 'uttarakhand', 'utter-pradesh', 'uttrakhand', 'west-bengal']


In [73]:
#list for store_location
store_locations = []
#checking store locations
for match in filtered_matches:
    store_locations.append(match)
print(len(store_locations))

39


In [74]:
#heaers for post requests
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Xstoreaccesskey": "pNRq8BGvIHk/G9AwlBxKG5lz97eYDaxnWO0YYs+VrMY="
}
#headers for post requests
payload = {
    "keyword": None,
    "range": 10,
    "perPage": 9999
}

In [75]:
# Iterate through each store location
for item in store_locations:
    # Update payload with the current store location
    payload["keyword"] = item
    
    # Endpoint URL for fetching store data
    url = 'https://locator-stores.lenskart.com/api/v3/store/list'
    
    try:
        # Send POST request to fetch store data
        res = requests.post(url, headers=header, data=payload)
        
        # Check if the request is successful
        if res.status_code == 200:
            # Construct file path for storing JSON response
            file_path = f"storeData/{item}.json"
            
            # Write JSON response to file
            with open(file_path, "w") as json_file:
                json_file.write(res.text)
        else:
            # Print message if request fails
            print(f"Failed to fetch data for {item}. Status code: {res.status_code}")
    except Exception as e:
        # Handle exceptions
        print(f"Error fetching data for {item}: {str(e)}")

In [77]:

# Define the field names for the CSV
fieldnames = ['id', 'alt_store_name_catch', 'address_full', 'address_state', 'store_phone', 'store_email',
              'business_hours_openi', 'business_hours_closi', 'averageRating', 'totalReviews','pincode']

# Initialize an empty list to store data rows
data_rows = []

# Iterate over each store location
for i in store_locations:
    with open(f'storeData/{i}.json', 'r') as fr:
        # Read JSON data
        json_data = json.load(fr)
        
        # Extract relevant data from JSON
        for store in json_data['data']['data']:
            store_data = {
                'id': store['id'],
                'alt_store_name_catch': store['alt_store_name_catch'],
                'address_full': store['address_full'],
                'address_state': store['address_state'],
                'store_phone': store['store_phone'],
                'store_email': store['store_email'],
                'business_hours_openi': store['business_hours_openi'],
                'business_hours_closi': store['business_hours_closi'],
                'averageRating': store['ratingAndReviewCount']['averageRating'],
                'totalReviews': store['ratingAndReviewCount']['totalReviews'],
                'pincode':store['address_pin_code']
            }
            # Append store data to the list
            data_rows.append(store_data)

# Write data to CSV file
with open('dataset/store_data.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write header
    writer.writeheader()
    
    # Write data rows
    for row in data_rows:
        writer.writerow(row)

print("CSV file created successfully.")


CSV file created successfully.
